# Deep Learning for NLP Best Practices [한글 번역]
http://ruder.io/deep-learning-nlp-best-practices/index.html#introduction

### Introduction

이 포스트는 자연어 처리 분야에서 신경망을 사용한 모범사례들을 모은 것입니다. 
<br>
새로운 인사이트가 있다면 정기적으로 업데이트 될것이며 지금도 진화하고 있는 NLP에서의 딥러닝을 따라잡기 위함입니다.
    
NLP커뮤니티에서 attention과 LSTM이 어떤 목적이든 간에 최고의 성능을 낸다는 농담이 있어왔습니다. 
<br>
비록 지난 2년간 진실이었지만, NLP 커뮤니티는 baseline에서 보다 흥미로운 모델로 서서히 이동하고 있습니다.
    
하지만, 우리가 앞으로의 2년에도 "다음" LSTM+attention을 발견하는 데에 시간을 쓰고 싶지않을 것입니다. 
<br>
이미 잘 동작하는 방법이나 트릭을 다시 내보이는걸 원하지도 않을거구요. 
<br>
수많은 딥러닝 라이브러리들이 이미 일반적인 신경망과 동작하는 best practice들(initialization schemes 등)을 인코딩하지만, 
<br>
특정 목적이나 도메인에 특화된 부분들은 실무자에게 맡겨져 있습니다.
    
이 포스트는 최신의 기술을 따라잡는걸 의미하는게 아닌, 광범위한 작업과 관련된 best practice들을 모으는 것입니다.
<br>
즉, 하나의 특정 아키텍쳐를 기술하는것보다 성공한 아키텍쳐들의 특징을 모으는 것이 목적입니다. 
<br>
이런 특징들의 다수가 최신기술을 구현하는데에 쓸만하겠지만..
<br>
이런 넓은 지식이 동작원리를 느끼는 직관을 예리하게 함으로써 
<br>
더 큰 진화와 전보다 의미있는 baseline과의 비교와 영감을 주도할 거라 믿습니다.
    
저는 당신이 NLP에 적용된 신경망에 친숙하다고 가정합니다.(만약, 그렇지 않다면 Yoav Goldberg의 [excellent primer](https://www.jair.org/media/4992/live-4992-9623-jair.pdf) 을 추천합니다.)
<br>
또한, 전반적으로든 특정 목적에 관해 NLP에 관심이 있다고도 가정하고 글을 씁니다. 
<br>
이 article의 주 목적은 당신이 가능한 빨리 의미있는 기여를 할 수있게끔 관련 우수사례들을 빨리 학습하게끔 하는 것입니다.

첫째로 대부분의 목적에 관련된 우수사례들의 개괄을 보여주고..
<br>
어느 목적이든 거의 공통적인 과제에 관련된 사례들의 윤곽을 잡아보려 합니다. 
<br>
예를 들자면, 특정 classification, sequence labelling, natural language generation, neural machine translation 등 말이죠.

*주: 모범사례로 뭔가를 다루는건 엄청나게 어렵습니다.
(뭐때문에 모범인데?, 더 나은 대안은 없나?... 등) 
<br>
이 포스트는 불완전한 저의 이해와 경험에 기반하여 작성했습니다.
<br>
앞으로 나올 내용에서 저는 최소한 다른 두 그룹 이상이 각각 독립적으로 이익인 것만을 사례로써 논해보고자합니다. 
<br>
각 모범사례마다 최소한 두개 이상의 레퍼런스를 쓰고자 노력해보겠습니다. 

## Best Practices

- Word Embedding
- Depth
- Layer Connections
- Dropout
- Multi-task learning
- Attention
- Optimization
- Ensembling
- Hyper-parameter optimization
- LSTM tricks

#### Word Embedding

Word Embedding은 분명 NLP의 최근 역사 중 가장 널리 알려진 모범사례입니다.
<br>
사전훈련된 embedding을 사용하면 Classification을 돕는 것으로 잘 알려져있습니다.[Yoon kim](http://arxiv.org/abs/1408.5882)
<br>
Word Embedding의 최적 차원은 task에 의존적입니다.(더 적은 차원은 개체명 인식(named entity recognition) 이나 품사태깅(POS tagging)에 잘 동작합니다.)
<br>
반면, 큰 차원은 sentiment analysis같은 의미론적 분석에 보다 유용합니다.

#### Depth

현재 최신의 접근법은 일반적으로 3~4층의 deep Bi-LSTM을 쓰는 것입니다.
<br>
(품사태깅(POS tagging)과 의미역결정(Semantic Role Labelling) 등..[reference](https://homes.cs.washington.edu/~luheng/files/acl2017_hllz.pdf))
<br>
어떤 목적에서의 모델은 좀 더 깊어질 수도 있습니다. 구글의 NMT모델은 8 인코더, 8 디코더 레이어를 가집니다.[reference](https://arxiv.org/abs/1609.08144)
<br>
그러나 대부분의 경우, 층을 2개 보다 깊은 모델을 만들어서 성능개선을 시기는 경우는 미미하다합니다.[reference](https://arxiv.org/pdf/1707.06799.pdf) 

이런 발견들은 대부분의 시퀀스 태깅과 구조화된 예측 문제에 적용됩니다.
<br>
분류문제에서 매우 깊은 모델은 오직 character 단위의 입력에서만 잘 동작하고 얕은 층의  단어 입력 모델은 여전히 최첨단 기술입니다.
<br>
[Zhang, X., Zhao, J., & LeCun, Y. (2015). Character-level Convolutional Networks for Text Classification.](https://arxiv.org/abs/1509.01626)
<br>
[Conneau, A., Schwenk, H., Barrault, L., & Lecun, Y. (2016). Very Deep Convolutional Networks for Natural Language Processing.](https://arxiv.org/abs/1606.01781)
<br>
[Le, H. T., Cerisara, C., & Denis, A. (2017). Do Convolutional Networks need to be Deep for Text Classification ?](https://arxiv.org/abs/1707.04108)

#### Layer Connections

딥뉴럴넷을 학습시킬때, Vanishing Gradient 문제를 피하기위해 필수적인 몇가지 트릭이 있습니다. 
<br>
다른 층과 연결하는 것이 제안됐었는데, 여기서 3가지를 논해보고자 합니다.

1. highway layers
2. residual connections
3. dence connections

**Highway layers** 
Highway layers([Srivastava et al., 2015](https://arxiv.org/pdf/1507.06228.pdf))는 LSTM의 게이트에서 착안되었습니다. 


입력 $\mathbf{x}$에 비선형성을 주는 $\mathbf{g}$로 affine변환이 적용된 히든레이어 1층의 MLP가 있다고 가정해봅시다. 

$\mathbf{h} = \mathbf{g}(\mathbf{W}\mathbf{x} + \mathbf{b})$ 

highway layer는 아래의 함수를 계산합니다.

$\mathbf{h} = \mathbf{t} \odot \mathbf{g}(\mathbf{W}\mathbf{x}+\mathbf{b}) + (1 - \mathbf{t}) \odot \mathbf{x}$

여기서 $\odot$은 element-wise 곱이며, $\mathbf{t}= \sigma(\mathbf{W}_T \mathbf{x} + \mathbf{b}_T)$ 는 
<br>
transform gate라 부르고 $(1-\mathbf{t})$는

In [1]:
import numpy as np

In [8]:
a = np.array([1,2,3])

a * a

array([1, 4, 9])